In [1]:
import tensorflow as tf
from PIL import Image
import os
import numpy as np
import extract_faces_keep_in_mem
from search_videos_in_directory import search_videos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

[INFO] loading model...
[INFO] loading model...


In [2]:
input_direc = input("Enter the absolute path of the input directory: ")
output_direc = input("Enter the absolute path of the output directory: ")
try:
    os.mkdir(output_direc)
except FileExistsError:
    pass
search_videos(input_direc, output_direc)

Enter the absolute path of the input directory: /home/debanik/downloaded_videos
Enter the absolute path of the output directory: /home/debanik/PycharmProjects/Detecting-Deepfakes/Face_images
Enter the number of images to be generated per video file: 1


In [3]:
for root, dirs, files in os.walk(output_direc):
    for file in files:

        try:
            im = Image.open(root + '/' + str(file))
            im.verify()
        except:
            print(str(file))
            os.remove(root + '/' + str(file))

In [ ]:
# input_direc = '/home/debanik/downloaded_videos'
# output_direc = '/home/debanik/PycharmProjects/Detecting-Deepfakes/Face_images'

In [ ]:
# create generator
datagen = ImageDataGenerator(samplewise_center=True, 
                             samplewise_std_normalization=True, 
                             rotation_range=10, 
                             horizontal_flip = True, 
                             validation_split=0.2)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(output_direc, class_mode='binary', batch_size=64)
# # confirm the iterator works
# batchX, batchy = train_it.next()
# print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [ ]:
x = Input(shape = (256, 256, 3))

x1 = Conv2D(16, (3, 3), strides = 1, padding='same', activation = 'relu')(x)
x1 = Conv2D(4, (1, 1), padding='same', activation = 'relu')(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D(pool_size=(8, 8), padding='same')(x1)

y = Flatten()(x1)
y = Dropout(0.5)(y)
y = Dense(1, activation = 'sigmoid')(y)

In [ ]:
model = Model(inputs = x, outputs = y)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit_generator(train_it, epochs = 100)